In [7]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2
from models.Fed import FedAvg
from models.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models.test import test_img


import math


# from sympy import * 
from utils.functions import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
class my_argument:    
    epochs = 200    #"rounds of training"
    num_users = 240  # "number of users: N"
    num_sel_users = 12
    frac = 0.5 #"the fraction of clients: C"
    local_ep=1 #"the number of local epochs: E"
    local_bs=50 #"local batch size: B"
    bs=50 #"test batch size"
    lr=0.01 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    weight_decay = 5e-4
    opt = 'SGD'
    loss = 'Cross'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='batch_norm' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='cifar' #, help="name of dataset")
    iid=1
    num_classes=10#, help="number of classes")
    num_channels=1#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose=False#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cuda" if use_cuda else "cpu")
print(args.device)

True
cuda


In [9]:
from torchvision import datasets, transforms


trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)

dict_users = cifar_iid(dataset_train, args.num_users)


Files already downloaded and verified
Files already downloaded and verified


In [10]:
N = args.num_users
K = args.num_sel_users

p_array = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
# p_array = np.array([0])

# print(p_matrix)
p_sel = np.random.randint(low=0, high=len(p_array), size=(N,))

p_per_user = np.ones((N,))

for i in range(N):
    p_per_user[i] = p_array[p_sel[i]]

print(p_per_user)

[0.3 0.5 0.1 0.5 0.3 0.1 0.5 0.3 0.1 0.3 0.3 0.2 0.5 0.3 0.3 0.4 0.1 0.3
 0.4 0.5 0.2 0.2 0.5 0.5 0.3 0.1 0.1 0.5 0.4 0.1 0.4 0.5 0.5 0.3 0.5 0.5
 0.3 0.4 0.5 0.1 0.5 0.5 0.4 0.4 0.4 0.5 0.3 0.4 0.4 0.5 0.4 0.1 0.5 0.1
 0.1 0.2 0.1 0.5 0.1 0.2 0.2 0.5 0.1 0.1 0.5 0.1 0.1 0.1 0.3 0.4 0.2 0.1
 0.2 0.3 0.5 0.2 0.5 0.1 0.3 0.4 0.2 0.3 0.5 0.1 0.4 0.5 0.2 0.2 0.2 0.3
 0.1 0.1 0.5 0.3 0.2 0.1 0.3 0.1 0.3 0.5 0.3 0.4 0.3 0.2 0.5 0.4 0.2 0.2
 0.2 0.1 0.1 0.3 0.3 0.3 0.3 0.1 0.4 0.4 0.2 0.2 0.2 0.1 0.4 0.4 0.2 0.2
 0.2 0.1 0.3 0.4 0.4 0.5 0.3 0.1 0.4 0.3 0.2 0.4 0.4 0.4 0.4 0.3 0.3 0.2
 0.1 0.1 0.4 0.2 0.1 0.1 0.4 0.1 0.5 0.2 0.1 0.5 0.3 0.3 0.4 0.2 0.3 0.1
 0.2 0.1 0.2 0.1 0.3 0.2 0.5 0.1 0.3 0.5 0.2 0.4 0.5 0.2 0.5 0.5 0.2 0.5
 0.5 0.1 0.4 0.1 0.2 0.4 0.3 0.4 0.2 0.2 0.5 0.4 0.5 0.2 0.4 0.2 0.3 0.1
 0.4 0.2 0.3 0.5 0.4 0.1 0.4 0.4 0.3 0.2 0.2 0.5 0.4 0.1 0.4 0.3 0.1 0.2
 0.4 0.5 0.2 0.2 0.5 0.3 0.4 0.5 0.2 0.3 0.2 0.2 0.5 0.4 0.4 0.2 0.5 0.4
 0.2 0.5 0.4 0.2 0.1 0.5]


# 1. Random Selection

In [11]:
from models.Nets import NIN,CNN_moderate, CNNCifar3

# p = 0.3
# N = 120
# K = 24

N_trials = 1
Max_iter = 1000

args.opt = 'SGD'

lr_array = [0.1]

args.local_ep = 1
args.local_bs = 50
args.weight_decay = 5e-4

acc_test_arr_random  = np.zeros((len(lr_array),N_trials,Max_iter))
loss_test_arr_random = np.zeros((len(lr_array),N_trials,Max_iter))

P_random = []

for lr_idx in range(len(lr_array)):
    
    for trial_idx in range(N_trials):
        
        args.lr       = lr_array[lr_idx]
        
        net_glob = CNNCifar3(args)
        
        net_glob = net_glob.cuda()
        
        print(net_glob)

        net_glob.train()
        
        P_random = []

        # copy weights
        w_glob = net_glob.state_dict()
        for iter in range(Max_iter): #args.epochs
            
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4
            
            # print('lr=',args.lr)
            
            w_locals, loss_locals = [], []

            ###############################
            # 0. Dropout Realization
            ###############################    
            
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]
            
            result = np.where(u == 1)
            drop_result = np.where(u == 0)

            ###############################
            # 1. Weighted Random Selection
            ###############################

            #idxs_users = np.random.choice(range(N), K, replace=False)
            
            idxs_users = np.random.choice(result[0], K, replace=False)


            p_tmp = np.zeros(N)
            p_tmp[idxs_users] = 1

            P_random.append(p_tmp)

        #     idxs_users = np.random.choice(range(N), K, replace=False)
            for idx in idxs_users:
        #         print(idx)
                local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                w_locals.append(copy.deepcopy(w))
                loss_locals.append(copy.deepcopy(loss))
            # update global weights
            w_glob = FedAvg(w_locals)

            # copy weight to net_glob
            net_glob.load_state_dict(w_glob)

            # print loss
            loss_avg = sum(loss_locals) / len(loss_locals)

            #loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_random[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_random[lr_idx][trial_idx][iter] = loss_test
            if iter % 10 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)
            
import pickle

file_name = './results/CIFAR_IID_LeNet_accuracy_Random_N'+ str(N)+'_K'+ str(K)+'_lr01_asymDrop'
pickle.dump(acc_test_arr_random, open(file_name, 'wb'), -1)
# pickle.dump(loss_test_arr_random, open('./results/CIFAR_NonIID_LeNet_accuracy_w_Random_N120_K12_asymDrop', 'wb'), -1)

CNNCifar3(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Round   0, Train average loss 2.302 Test accuracy 14.080
Round  10, Train average loss 2.206 Test accuracy 25.420
Round  20, Train average loss 2.014 Test accuracy 28.660
Round  30, Train average loss 1.930 Test accuracy 32.940
Round  40, Train average loss 1.882 Test accuracy 36.000
Round  50, Train average loss 1.790 Test accuracy 39.990
Round  60, Train average loss 1.755 Test accuracy 40.860
Round  70, Train average loss 1.750 Test accuracy 42.780
Round  80, Train average loss 1.702 Test accuracy 45.270
Round  90, Train average loss 1.637 Test acc

# 2. Weighted Random Selection

In [12]:
from models.Nets import NIN,CNN_moderate, CNNCifar3

lr_array = [0.1]


args.local_ep = 1
args.local_bs = 50
args.weight_decay = 5e-4

# N = 120
# K = 24

N_trials = 1
Max_iter = 1000

args.opt = 'SGD'

args.local_ep = 1
args.local_bs = 50


acc_test_arr_w_random_asymDrop  = np.zeros((len(lr_array), N_trials, Max_iter))
loss_test_arr_w_random_asymDrop = np.zeros((len(lr_array), N_trials, Max_iter))


for trial_idx in range(N_trials):
    
    for lr_idx in range(len(lr_array)):
        
        args.lr = lr_array[lr_idx]
        
        P_w_random_asymDrop = []
        
        print()
        print('Learning Rate =',args.lr)
        print()
        net_glob = CNNCifar3(args)
        net_glob = net_glob.cuda()
        print(net_glob)

        net_glob.train()

        # copy weights
        w_glob = net_glob.state_dict()
        for iter in range(Max_iter): #args.epochs
            w_locals, loss_locals = [], []
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4
                
                
            ###############################
            # 0. Dropout Realization
            ###############################    
            
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]
            
            result = np.where(u == 1)
            drop_result = np.where(u == 0)

            ###############################
            # 1. Weighted Random Selection
            ###############################

            if iter == 0:
                idxs_users = np.random.choice(result[0], K, replace=False)
#                 print('select=',select)
            else:
                P = np.array(P_w_random_asymDrop)
                P_sum = np.sum(P, axis=0).astype(int)
        
                for i in drop_result[0]:
                    P_sum[i] = Max_iter + 1   

                P_sum_sort = P_sum.argsort()

                idxs_users = P_sum_sort[:K]
            

            p_tmp = np.zeros(N)
            p_tmp[idxs_users] = 1

            P_w_random_asymDrop.append(p_tmp)


        #     idxs_users = np.random.choice(range(N), K, replace=False)
            for idx in idxs_users:
        #         print(idx)
                local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                w_locals.append(copy.deepcopy(w))
                loss_locals.append(copy.deepcopy(loss))
            # update global weights
            w_glob = FedAvg(w_locals)

            # copy weight to net_glob
            net_glob.load_state_dict(w_glob)

            # print loss
            loss_avg = sum(loss_locals) / len(loss_locals)

    #         loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_w_random_asymDrop[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_w_random_asymDrop[lr_idx][trial_idx][iter] = loss_test
            if iter % 10 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)


file_name = './results/CIFAR_IID_LeNet_accuracy_w_Random_N'+ str(N)+'_K'+ str(K)+'_lr01_asymDrop'
pickle.dump(acc_test_arr_w_random_asymDrop, open(file_name, 'wb'), -1)


Learning Rate = 0.1

CNNCifar3(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Round   0, Train average loss 2.305 Test accuracy 12.000
Round  10, Train average loss 2.122 Test accuracy 25.490
Round  20, Train average loss 2.010 Test accuracy 30.000
Round  30, Train average loss 1.899 Test accuracy 32.880
Round  40, Train average loss 1.860 Test accuracy 36.650
Round  50, Train average loss 1.818 Test accuracy 39.000
Round  60, Train average loss 1.767 Test accuracy 41.630
Round  70, Train average loss 1.702 Test accuracy 41.710
Round  80, Train average loss 1.699 Test accuracy 42.000
Round  90, Train avera

In [14]:
file_name = './results/CIFAR_IID_LeNet_accuracy_w_Random_N'+ str(N)+'_K'+ str(K)+'_lr01_asymDrop'
pickle.dump(acc_test_arr_w_random_asymDrop, open(file_name, 'wb'), -1)

# 2. Partition

In [13]:
# N   = 120
# K   = 24

Codebook_Proposed_T6 = CodeBookDesign_Bin(N,K,6)
Codebook_Proposed_T4 = CodeBookDesign_Bin(N,K,4)
Codebook_Proposed_T3 = CodeBookDesign_Bin(N,K,3)

Codebook_Partition = CodeBookDesign_Partition(N,K)

# print(Codebook_Partition)

@CodeBookDesign_Bin, Codebook Size= 780
@CodeBookDesign_Bin, Codebook Size= 34220
@CodeBookDesign_Bin, Codebook Size= 1581580
@CodeBookDesign_Partition, Codebook Size= 20


In [15]:
from models.Nets import NIN,CNN_moderate, CNNCifar3

lr_array = [0.1]


args.local_ep = 1
args.local_bs = 50
args.weight_decay = 5e-4

# N = 120
# K = 24

N_trials = 1
Max_iter = 1000

args.opt = 'SGD'

args.local_ep = 1
args.local_bs = 50

acc_test_arr_Partition_N120_K12_asymDrop  = np.zeros((len(lr_array), N_trials, Max_iter))
loss_test_arr_Partition_N120_K12_asymDrop = np.zeros((len(lr_array), N_trials, Max_iter))


is_selection_applied = True
is_print = False



for trial_idx in range(N_trials):
    
    for lr_idx in range(len(lr_array)):
        
        args.lr = lr_array[lr_idx]
        
        P_partition_asymDrop = []
        
        print()
        print('Learning Rate =',args.lr)
        print()
        net_glob = CNNCifar3(args)
        net_glob = net_glob.cuda()
        print(net_glob)

        net_glob.train()

        # copy weights
        w_glob = net_glob.state_dict()
        
        for iter in range(Max_iter): #args.epochs
            w_locals, loss_locals = [], []
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4

        #     u = np.random.binomial(1, 1-p, size=(N))
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]

            tmp = Codebook_Partition * u

            codebook_en = 1*(np.sum(tmp,axis=1) == K)

            if np.sum(codebook_en) == 0:
                if is_print:
                    print('[iter=',iter,'] @Partition, there is no available codebook')
                p_tmp = np.zeros(N)
            else:
                if is_print:
                    print('[iter=',iter,'] @Partition, Codebook selection')
                idx_en = np.where(codebook_en == 1)

                if is_selection_applied:
                    codebook_en_num = np.sum(codebook_en)
                    Codebook_En = np.reshape(Codebook_Partition[idx_en,:],(codebook_en_num,N))

                    p_tmp = UserSelection_Codebook(P_partition_asymDrop, Codebook_En)
                else:
                    idx_sel = np.random.choice(idx_en[0], 1, replace=False)
                    p_tmp = Codebook_Partition[idx_sel[0],:]

        #                 idx_sel = np.random.choice(idx_en[0], 1, replace=False)

        #                 p_tmp = Codebook_Partition[idx_sel[0],:]

            idxs_users = np.where(p_tmp == 1)

            P_partition_asymDrop.append(p_tmp)

        #     idxs_users = np.random.choice(range(args.num_users), K, replace=False)

            if np.sum(p_tmp) == K:
                for idx in idxs_users[0]:
                    print('selected idx=',idx)
                    local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                    w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                    w_locals.append(copy.deepcopy(w))
                    loss_locals.append(copy.deepcopy(loss))
                # update global weights
                w_glob = FedAvg(w_locals)

                # copy weight to net_glob
                net_glob.load_state_dict(w_glob)

                # print loss
                loss_avg = sum(loss_locals) / len(loss_locals)

        #         loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_Partition_N120_K12_asymDrop[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_Partition_N120_K12_asymDrop[lr_idx][trial_idx][iter] =loss_test
            if iter % 10 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)
            
file_name = './results/CIFAR_IID_LeNet_accuracy_Partition_N'+ str(N)+'_K'+ str(K)+'_lr01_asymDrop'
pickle.dump(acc_test_arr_Partition_N120_K12_asymDrop, open(file_name, 'wb'), -1)

# pickle.dump(acc_test_arr_Partition_N120_K12_asymDrop, open('./results/CIFAR_IID_LeNet_accuracy_Partition_N120_K24_lr01_asymDrop', 'wb'), -1)


Learning Rate = 0.1

CNNCifar3(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Round   0, Train average loss 1.003 Test accuracy 9.570
selected idx= 156
selected idx= 157
selected idx= 158
selected idx= 159
selected idx= 160
selected idx= 161
selected idx= 162
selected idx= 163
selected idx= 164
selected idx= 165
selected idx= 166
selected idx= 167
Round  10, Train average loss 2.308 Test accuracy 9.480
selected idx= 156
selected idx= 157
selected idx= 158
selected idx= 159
selected idx= 160
selected idx= 161
selected idx= 162
selected idx= 163
selected idx= 164
selected idx= 165
selected idx= 166
selected 

# 3. MultiRound-SecAgg (T=6)

In [21]:
from models.Nets import NIN,CNN_moderate, CNNCifar3

lr_array = [0.1]


args.local_ep = 1
args.local_bs = 50
args.weight_decay = 5e-4

# N = 120
# K = 24

N_trials = 1
Max_iter = 1000

args.opt = 'SGD'

args.local_ep = 1
args.local_bs = 50

acc_test_arr_Proposed_N120_K12_T6_asymDrop  = np.zeros((len(lr_array), N_trials, Max_iter))
loss_test_arr_Proposed_N120_K12_T6_asymDrop = np.zeros((len(lr_array), N_trials, Max_iter))


is_selection_applied = True
is_print = True



for trial_idx in range(N_trials):
    
    for lr_idx in range(len(lr_array)):
        
        args.lr = lr_array[lr_idx]
        
        P_proposed_T6_asymDrop = []
        
        print()
        print('Learning Rate =',args.lr)
        print()
        net_glob = CNNCifar3(args)
        net_glob = net_glob.cuda()
        print(net_glob)

        net_glob.train()

        # copy weights
        w_glob = net_glob.state_dict()
        
        for iter in range(Max_iter): #args.epochs
            w_locals, loss_locals = [], []
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4

        #     u = np.random.binomial(1, 1-p, size=(N))
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]

            tmp = Codebook_Proposed_T6 * u

            codebook_en = 1*(np.sum(tmp,axis=1) == K)

            if np.sum(codebook_en) == 0:
                if is_print:
                    print('[iter=',iter,'] @Proposed T=6, there is no available codebook')
                p_tmp = np.zeros(N)
            else:
                if is_print:
                    print('[iter=',iter,'] @Proposed T=6, Codebook selection')
                idx_en = np.where(codebook_en == 1)

                if is_selection_applied:
                    codebook_en_num = np.sum(codebook_en)
                    Codebook_En = np.reshape(Codebook_Proposed_T6[idx_en,:],(codebook_en_num,N))

                    p_tmp = UserSelection_Codebook(P_proposed_T6_asymDrop, Codebook_En)
                else:
                    idx_sel = np.random.choice(idx_en[0], 1, replace=False)
                    p_tmp = Codebook_Proposed_T6[idx_sel[0],:]

        #                 idx_sel = np.random.choice(idx_en[0], 1, replace=False)

        #                 p_tmp = Codebook_Partition[idx_sel[0],:]

            idxs_users = np.where(p_tmp == 1)

            P_proposed_T6_asymDrop.append(p_tmp)

        #     idxs_users = np.random.choice(range(args.num_users), K, replace=False)

            if np.sum(p_tmp) == K:
                for idx in idxs_users[0]:
                    print('selected idx=',idx)
                    local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                    w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                    w_locals.append(copy.deepcopy(w))
                    loss_locals.append(copy.deepcopy(loss))
                # update global weights
                w_glob = FedAvg(w_locals)

                # copy weight to net_glob
                net_glob.load_state_dict(w_glob)

                # print loss
                loss_avg = sum(loss_locals) / len(loss_locals)

        #         loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_Proposed_N120_K12_T6_asymDrop[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_Proposed_N120_K12_T6_asymDrop[lr_idx][trial_idx][iter] =loss_test
            if iter % 10 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)

file_name = './results/CIFAR_IID_LeNet_accuracy_Proposed_N'+ str(N)+'_K'+ str(K)+'_T6_lr01_asymDrop'
pickle.dump(acc_test_arr_Proposed_N120_K12_T6_asymDrop, open(file_name, 'wb'), -1)
            
# pickle.dump(acc_test_arr_Proposed_N120_K12_T6_asymDrop, open('./results/CIFAR_IID_LeNet_accuracy_Proposed_N120_K24_T6_lr01_asymDrop', 'wb'), -1)


Learning Rate = 0.1

CNNCifar3(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
[iter= 0 ] @Proposed T=6, there is no available codebook
Round   0, Train average loss 2.308 Test accuracy 10.160
[iter= 1 ] @Proposed T=6, Codebook selection
selected idx= 48
selected idx= 49
selected idx= 50
selected idx= 51
selected idx= 52
selected idx= 53
selected idx= 54
selected idx= 55
selected idx= 56
selected idx= 57
selected idx= 58
selected idx= 59
selected idx= 84
selected idx= 85
selected idx= 86
selected idx= 87
selected idx= 88
selected idx= 89
selected idx= 102
selected idx= 103
selected idx= 104
selected idx= 10

# 4. MultiRound-SecAgg (T=4)

In [22]:
from models.Nets import NIN,CNN_moderate, CNNCifar3

lr_array = [0.1]


args.local_ep = 1
args.local_bs = 50
args.weight_decay = 5e-4

N = 120
K = 24

N_trials = 3
Max_iter = 1000

args.opt = 'SGD'

args.local_ep = 1
args.local_bs = 50

acc_test_arr_Proposed_N120_K12_T4_asymDrop  = np.zeros((len(lr_array), N_trials, Max_iter))
loss_test_arr_Proposed_N120_K12_T4_asymDrop = np.zeros((len(lr_array), N_trials, Max_iter))


is_selection_applied = True
is_print = True



for trial_idx in range(N_trials):
    
    for lr_idx in range(len(lr_array)):
        
        args.lr = lr_array[lr_idx]
        
        P_proposed_T4_asymDrop = []
        
        print()
        print('Learning Rate =',args.lr)
        print()
        net_glob = CNNCifar3(args)
        net_glob = net_glob.cuda()
        print(net_glob)

        net_glob.train()

        # copy weights
        w_glob = net_glob.state_dict()
        
        for iter in range(Max_iter): #args.epochs
            w_locals, loss_locals = [], []
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4

        #     u = np.random.binomial(1, 1-p, size=(N))
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]

            tmp = Codebook_Proposed_T4 * u

            codebook_en = 1*(np.sum(tmp,axis=1) == K)

            if np.sum(codebook_en) == 0:
                if is_print:
                    print('[iter=',iter,'] @Proposed T=4, there is no available codebook')
                p_tmp = np.zeros(N)
            else:
                if is_print:
                    print('[iter=',iter,'] @Proposed T=4, Codebook selection')
                idx_en = np.where(codebook_en == 1)

                if is_selection_applied:
                    codebook_en_num = np.sum(codebook_en)
                    Codebook_En = np.reshape(Codebook_Proposed_T4[idx_en,:],(codebook_en_num,N))

                    p_tmp = UserSelection_Codebook(P_proposed_T4_asymDrop, Codebook_En)
                else:
                    idx_sel = np.random.choice(idx_en[0], 1, replace=False)
                    p_tmp = Codebook_Proposed_T4[idx_sel[0],:]

        #                 idx_sel = np.random.choice(idx_en[0], 1, replace=False)

        #                 p_tmp = Codebook_Partition[idx_sel[0],:]

            idxs_users = np.where(p_tmp == 1)

            P_proposed_T4_asymDrop.append(p_tmp)

        #     idxs_users = np.random.choice(range(args.num_users), K, replace=False)

            if np.sum(p_tmp) == K:
                for idx in idxs_users[0]:
                    print('selected idx=',idx)
                    local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                    w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                    w_locals.append(copy.deepcopy(w))
                    loss_locals.append(copy.deepcopy(loss))
                # update global weights
                w_glob = FedAvg(w_locals)

                # copy weight to net_glob
                net_glob.load_state_dict(w_glob)

                # print loss
                loss_avg = sum(loss_locals) / len(loss_locals)

        #         loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_Proposed_N120_K12_T4_asymDrop[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_Proposed_N120_K12_T4_asymDrop[lr_idx][trial_idx][iter] =loss_test
            if iter % 10 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)
            
file_name = './results/CIFAR_IID_LeNet_accuracy_Proposed_N'+ str(N)+'_K'+ str(K)+'_T4_lr01_asymDrop'
pickle.dump(acc_test_arr_Proposed_N120_K12_T4_asymDrop, open(file_name, 'wb'), -1)


Learning Rate = 0.1

CNNCifar3(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
[iter= 0 ] @Proposed T=4, there is no available codebook
Round   0, Train average loss 1.342 Test accuracy 9.980
[iter= 1 ] @Proposed T=4, Codebook selection
selected idx= 0
selected idx= 1
selected idx= 2
selected idx= 3
selected idx= 8
selected idx= 9
selected idx= 10
selected idx= 11
selected idx= 16
selected idx= 17
selected idx= 18
selected idx= 19
selected idx= 24
selected idx= 25
selected idx= 26
selected idx= 27
selected idx= 52
selected idx= 53
selected idx= 54
selected idx= 55
selected idx= 68
selected idx= 69
selected 

# 5. MultiRound-SecAgg (T=3)

In [5]:
from models.Nets import NIN,CNN_moderate, CNNCifar3

lr_array = [0.1]


args.local_ep = 1
args.local_bs = 50
args.weight_decay = 5e-4

N = 120
K = 24

N_trials = 1
Max_iter = 1000

args.opt = 'SGD'

args.local_ep = 1
args.local_bs = 50


acc_test_arr_w_random_asymDrop  = np.zeros((len(lr_array), N_trials, Max_iter))
loss_test_arr_w_random_asymDrop = np.zeros((len(lr_array), N_trials, Max_iter))


for trial_idx in range(N_trials):
    
    for lr_idx in range(len(lr_array)):
        
        args.lr = lr_array[lr_idx]
        
        P_w_random_asymDrop = []
        
        print()
        print('Learning Rate =',args.lr)
        print()
        net_glob = CNNCifar3(args)
        net_glob = net_glob.cuda()
        print(net_glob)

        net_glob.train()

        # copy weights
        w_glob = net_glob.state_dict()
        for iter in range(Max_iter): #args.epochs
            w_locals, loss_locals = [], []
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4
                
                
            ###############################
            # 0. Dropout Realization
            ###############################    
            
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]
            
            result = np.where(u == 1)
            drop_result = np.where(u == 0)

            ###############################
            # 1. Weighted Random Selection
            ###############################

            if iter == 0:
                idxs_users = np.random.choice(result[0], K, replace=False)
#                 print('select=',select)
            else:
                P = np.array(P_w_random_asymDrop)
                P_sum = np.sum(P, axis=0).astype(int)
        
                for i in drop_result[0]:
                    P_sum[i] = Max_iter + 1   

                P_sum_sort = P_sum.argsort()

                idxs_users = P_sum_sort[:K]
            

            p_tmp = np.zeros(N)
            p_tmp[idxs_users] = 1

            P_w_random_asymDrop.append(p_tmp)


        #     idxs_users = np.random.choice(range(N), K, replace=False)
            for idx in idxs_users:
        #         print(idx)
                local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                w_locals.append(copy.deepcopy(w))
                loss_locals.append(copy.deepcopy(loss))
            # update global weights
            w_glob = FedAvg(w_locals)

            # copy weight to net_glob
            net_glob.load_state_dict(w_glob)

            # print loss
            loss_avg = sum(loss_locals) / len(loss_locals)

    #         loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_w_random_asymDrop[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_w_random_asymDrop[lr_idx][trial_idx][iter] = loss_test
            if iter % 10 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)
            



Learning Rate = 0.1

CNNCifar3(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


c:\ProgramData\Anaconda3\envs\env_py38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Round   0, Train average loss 2.305 Test accuracy 9.140
Round  10, Train average loss 2.083 Test accuracy 29.900
Round  20, Train average loss 1.910 Test accuracy 36.040
Round  30, Train average loss 1.820 Test accuracy 39.490
Round  40, Train average loss 1.761 Test accuracy 41.540
Round  50, Train average loss 1.690 Test accuracy 43.940
Round  60, Train average loss 1.645 Test accuracy 45.670
Round  70, Train average loss 1.603 Test accuracy 47.430
Round  80, Train average loss 1.557 Test accuracy 47.710
Round  90, Train average loss 1.548 Test accuracy 49.200
Round 100, Train average loss 1.514 Test accuracy 50.380
Round 110, Train average loss 1.503 Test accuracy 50.770
Round 120, Train average loss 1.452 Test accuracy 51.630
Round 130, Train average loss 1.447 Test accuracy 52.530
Round 140, Train average loss 1.416 Test accuracy 52.620
Round 150, Train average loss 1.398 Test accuracy 53.180
Round 160, Train average loss 1.385 Test accuracy 53.800
Round 170, Train average loss 1.

NameError: name 'pickle' is not defined

In [6]:
import pickle

pickle.dump (acc_test_arr_w_random_asymDrop, open('./results/CIFAR_IID_LeNet_accuracy_Proposed_N120_K24_T3_lr01_asymDrop', 'wb'), -1)